## Collect and Visualize NWM Forecasts

**Authors**: Anthony Castronova (acastronova@cuahsi.org), Irene Garousi-Nejad (igarousi@cuahsi.org)  <br>
**Last updated**: 

This notebook demonstrates how to collect and visualize NWM forecast data. All data used in this notebook are publicly acessible via Amazon AWS. Streamflow data is collected from operational NWM medium-range forecasts (https://console.cloud.google.com/storage/browser/national-water-model/).

This work was funded by:

 <img src="https://www.hydroshare.org/resource/dc269e23ff494a06b7372bc6034a5de2/data/contents/v3-SNOW/logo-img/CUAHSI-4-color-logo_with_URL.png" width="300" height="300" style="padding-right:50px"> <img src="https://www.hydroshare.org/resource/dc269e23ff494a06b7372bc6034a5de2/data/contents/v3-SNOW/logo-img/CIROHLogo_200x200.png" width="100" height="20">    

In [ ]:
import os
os.environ['USE_PYGEOS'] = '0'

import sys
import zarr
import numpy
import xarray
import rioxarray
import geopandas
import numpy as np
import pandas as pd
from typing import Dict
from pathlib import Path
from scipy import interpolate
import matplotlib.pyplot as plt
from dask.distributed import Client
from datetime import datetime, timedelta
from geocube.api.core import make_geocube

# Import a utility library for collecting medium-range forecast data.
import forecast

In [ ]:
# use a try accept loop so we only instantiate the client
# if it doesn't already exist.
try:
    print(client.dashboard_link)
except:    
    # The client should be customized to your workstation resources.
    # Machine size -> 52 GB, 16 CPU
    #client = Client()
    client = Client(n_workers=6, memory_limit='4GB') # per worker
    print(client.dashboard_link)

Define a feature of interest using an NHD+ Reach ID

In [ ]:
# NHD+ feature id of interest
nhd_feature_id=4966269

Collect streamflow for this `reach_id` using the `forecast.py` forecast utility.

In [ ]:
%%time

destination_folder = ".cache"
forecast_mode = 'medium_range_blend'
init_times = [0] 

# collect data for the reach above for multiple days
start_date = datetime(2024, 4, 1)
end_date = datetime(2024, 4, 3)
delta = timedelta(days=1)
dats = {}
while start_date <= end_date:
    ds = mediumrange.get_streamflow_for_reaches(start_date, init_times=init_times, reach_ids=[nhd_feature_id],
                                                forecast_mode=forecast_mode,
                                                merge_files=True,
                                                merge_format='Zarr',
                                                clean_on_success=False,
                                                )
    start_date += delta
    dats.update(ds)

In [ ]:
list(dats.keys())

In [ ]:
fig, ax = plt.subplots(1,1)

for k, dat in dats.items():
    print(f'plotting {k}')
    dat.streamflow.plot(ax=ax, label=k);
plt.grid()
plt.legend()

In [ ]:
%%time

#bucket_name = "national-water-model"
#wildcard = "nwm.t00z*channel_rt*"

destination_folder = ".cache"
forecast_mode = 'short_range'
init_times = range(0, 24)
# collect data for the reach above for multiple days
start_date = datetime(2024, 4, 1)
end_date = datetime(2024, 4, 4)
delta = timedelta(days=1)
dats = {}
while start_date <= end_date:
    ds = mediumrange.get_streamflow_for_reaches(start_date, init_times=init_times, reach_ids=[nhd_feature_id],
                                                forecast_mode=forecast_mode,
                                                merge_files=True,
                                                merge_format='Zarr',
                                                clean_on_success=False,
                                                )
    start_date += delta
    dats.update(ds)

In [ ]:
list(dats.keys())

In [ ]:
from matplotlib import colormaps, cm
import numpy as np

grey_cm = colormaps['Blues']
colors = grey_cm(np.linspace(0, 1, 24))

fig, ax = plt.subplots(1,1)

i = 0
for key, dat in dats.items():
    
    dat.streamflow.plot(ax=ax, color=colors[i]);

    i += 1
    if i > 23:
        break
plt.grid()